<a href="https://colab.research.google.com/github/Swathi1309/Medical-Image-Analysis/blob/main/ED18B034_ED6001_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [24]:
import cv2
import numpy as np
from math import floor, ceil, log10, sqrt, radians, cos, sin
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from statistics import median

# Supporting Functions

## Image Transformations

In [25]:
# Function to translate image using bilinear interpolation
def translate(image, x_shift, y_shift):
  new_image = np.zeros(image.shape)

  x_int = floor(x_shift)
  x_dec = x_shift - x_int
  x_dec = round(x_dec, 1)
  
  y_int = floor(y_shift)
  y_dec = y_shift - y_int
  y_dec = round(y_dec, 1)

  a_weight = x_dec*y_dec
  b_weight = (1-x_dec)*y_dec
  c_weight = (x_dec)*(1-y_dec)
  d_weight = (1-x_dec)*(1-y_dec)

  for i in range(image.shape[0]):
    for j in range(image.shape[1]):
      if i>=y_int and j>=x_int:
        a = image[i-1-y_int, j-1-x_int] if (i-1>=y_int and j-1>=x_int) else 0
        b = image[i-y_int, j-1-x_int] if (j-1>=x_int) else 0
        c = image[i-1-y_int, j-x_int] if (i-1>=y_int) else 0
        d = image[i-y_int, j-x_int]
        new_image[i,j] = a*a_weight + b*b_weight + c*c_weight + d*d_weight
  return new_image


# Function to rotate image using bilinear interpolation
def rotate(image, angle):
  new_image = np.zeros(image.shape)
  angle = radians(angle)
  inv_transformation = np.linalg.inv(np.array([[cos(angle), sin(angle)],[-1*sin(angle), cos(angle)]]))
  for i in range (image.shape[0]):
    for j in range (image.shape[1]):
      x_inv = i - 512.5
      y_inv = j - 512.5
      [[x], [y]] = np.matmul(inv_transformation, np.array([[x_inv],[y_inv]]))
      x+=512.5
      y+=512.5
      if x<0 or x>=1024 or y<0 or y>=1024:
        new_image[i, j] = 0
      else:
        a = image[floor(x), floor(y)]
        b = image[floor(x)+1, floor(y)] if floor(x)+1<1024 else 0
        c = image[floor(x), floor(y)+1] if floor(y)+1<1024 else 0
        d = image[floor(x)+1, floor(y)+1] if (floor(x)+1<1024 and floor(y)+1<1024) else 0
        x_dec = round(x - floor(x), 1)
        y_dec = round(y - floor(y), 1)
        a_weight = (1-x_dec)*(1-y_dec)
        b_weight = (1-x_dec)*y_dec
        c_weight = (x_dec)*(1-y_dec)
        d_weight = x_dec*y_dec
        new_image[i, j] = a_weight*a + b_weight*b + c_weight*c + d_weight*d

  return new_image

# Function to scale image using bilinear interpolation
def scale(image, factor):
  x_dim = floor(image.shape[0]*factor)
  y_dim = floor(image.shape[1]*factor)
  new_image = np.zeros((x_dim, y_dim))
  for i in range(x_dim):
    for j in range(y_dim):
      x = i/factor
      y = j/factor
      if x<0 or x>=1024 or y<0 or y>=1024:
        new_image[i, j] = 0
      else:
        a = image[floor(x), floor(y)]
        b = image[floor(x)+1, floor(y)] if floor(x)+1<1024 else 0
        c = image[floor(x), floor(y)+1] if floor(y)+1<1024 else 0
        d = image[floor(x)+1, floor(y)+1] if (floor(x)+1<1024 and floor(y)+1<1024) else 0
        x_dec = round(x - floor(x), 1)
        y_dec = round(y - floor(y), 1)
        a_weight = (1-x_dec)*(1-y_dec)
        b_weight = (1-x_dec)*y_dec
        c_weight = (x_dec)*(1-y_dec)
        d_weight = x_dec*y_dec
        new_image[i, j] = a_weight*a + b_weight*b + c_weight*c + d_weight*d
  return new_image

## Histogram Equalization

In [26]:
# Function to get the PDF of the given image
def histogram(image):
  intensity_arr = [0 for _ in range(int(np.max(image))+1)]
  for i in range(image.shape[0]):
    for j in range(image.shape[1]):
      intensity = floor(image[i,j])
      intensity_arr[intensity] +=1
  return intensity_arr

# Function to get the clipped PDF of the given image
def clipped_histogram(image, limit):
  intensity_arr = histogram(image)
  lt = floor(limit*max(intensity_arr))
  sum = 0
  for i in range(len(intensity_arr)):
    if intensity_arr[i]>lt:
      sum += (intensity_arr[i]-lt)
      intensity_arr[i] = lt
  extra = floor(sum/len(intensity_arr))
  for i in range(len(intensity_arr)):
    intensity_arr[i]+=extra
  return intensity_arr

# Function to convert the PDF into a Cumulative Density Function
def get_cdf(arr):
  arr_new = [0 for _ in range(len(arr))]
  arr_new[0] = arr[0]
  for i in range(1, len(arr_new)):
    arr_new[i] =arr_new[i-1] +arr[i]
  return arr_new

In [27]:
# Function for improving contrast of an image, given the CDF
def hist_equalization(image, cdf_intensities):
  cdf_min = min(cdf_intensities)
  if cdf_min == image.shape[0]**2:
    den = 1
  else:
    den = (image.shape[0]**2 - cdf_min)
  new_image = np.zeros(image.shape)
  for i in range(image.shape[0]):
    for j in range(image.shape[1]):
      intensity = floor(image[i,j])
      new_intensity = (cdf_intensities[intensity]-cdf_min)*(len(cdf_intensities)-1)/den
      new_image[i,j] = int(floor(new_intensity))
  return new_image

# Function to enhance contrast of an image by Contrast Limited Adaptive Histogram Equalization
def clahe(image, filter_size, limit):
  new_image = np.zeros(image.shape)
  center = int((filter_size-1)/2)
  for i in range(image.shape[0]):
    for j in range(image.shape[1]):
      neig = get_neighbourhood(original_image, i, j, filter_size)
      intensities = clipped_histogram(neig, limit)
      cdf = get_cdf(intensities)
      hist_eq = hist_equalization(neig, cdf)
      intensity = hist_eq[center,center]
      new_image[i,j] = intensity
  return new_image

## Noise and Filters

In [28]:
# Function to add noise to image
def add_noise(image, noise_type, variance):

  if noise_type=="salt_and_pepper":
    row, col = image.shape
    noisy = np.copy(image)
    s_vs_p = 0.5
    amount = variance / (2 * 255 * 255 / 4)
    # White spots
    num_salt = np.ceil(amount * image.size * s_vs_p)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
    for i in range(len(coords[0])):
      x, y = coords[0][i], coords[1][i]
      noisy[x,y] = 255
    # Black spots
    num_pepper = np.ceil(amount* image.size * (1 - s_vs_p))
    coords = [np.random.randint(0, i - 1, int(num_pepper))for i in image.shape]
    for i in range(len(coords[0])):
      x, y = coords[0][i], coords[1][i]
      noisy[x,y] = 0
    return noisy

  elif noise_type == "gaussian":
    row,col = image.shape
    mean = 0
    sigma = variance**0.5
    gauss = np.random.normal(mean,sigma,(row,col))
    gauss = gauss.reshape(row,col)
    noisy = image + gauss
    return noisy

In [29]:
# Function to get the neighbourhood of a given pixel in the image
def get_neighbourhood(image, i, j, n):
  neighbourhood = np.array([[0 for _ in range(n)] for _ in range(n)])
  center = (n-1)/2
  for a in range(n):
    for b in range(n):
      x = b-center
      y = a-center
      # x_ and y_ are the pixels of the actual image
      x_ = i + x
      y_ = j + y
      if x_<0:
        x_ = (-1*x_) -1
      if y_<0:
        y_ = (-1*y_) -1
      if x_>=image.shape[0]:
        x_ = image.shape[0] - (x_ - (image.shape[0]-1))
      if y_>=image.shape[1]:
        y_ = image.shape[1] - (y_ - (image.shape[1]-1))
      neighbourhood[a,b] = image[int(x_), int(y_)]
  return neighbourhood

# Function to get the 4-n or 8-n mean of an given pixel in the image
def get_mean(i, j, image, n):
  total = 0
  if i-1>=0:
    total+=image[i-1, j]
  if j-1>=0:
    total+=image[i, j-1]
  if i+1<image.shape[0]:
    total+=image[i+1, j]
  if j+1<image.shape[1]:
    total+=image[i, j+1]
  if i-1>=0 and j-1>=0 and n==8:
    total+=image[i-1, j-1]
  if i+1<image.shape[0] and j-1>=0 and n==8:
    total+=image[i+1, j-1]
  if i+1<image.shape[0] and j+1<image.shape[1] and n==8:
    total+=image[i+1, j+1]
  if i-1>=0 and j+1<image.shape[1] and n==8:
    total+=image[i-1, j+1]
  mean = floor(total/n)
  return mean

# Function to get the 4-n or 8-n median of an given pixel in the image
def get_median(i, j, image, n):
  arr = []
  if i-1>=0:
    arr.append(image[i-1, j])
  if j-1>=0:
    arr.append(image[i, j-1])
  if i+1<image.shape[0]:
    arr.append(image[i+1, j])
  if j+1<image.shape[1]:
    arr.append(image[i, j+1])
  if i-1>=0 and j-1>=0 and n==8:
    arr.append(image[i-1, j-1])
  if i+1<image.shape[0] and j-1>=0 and n==8:
    arr.append(image[i+1, j-1])
  if i+1<image.shape[0] and j+1<image.shape[1] and n==8:
    arr.append(image[i+1, j+1])
  if i-1>=0 and j+1<image.shape[1] and n==8:
    arr.append(image[i-1, j+1])
  return floor(median(arr))

In [30]:
# Function to apply mean filter on an image
def mean_filter(image, n):
  filtered = np.zeros(image.shape)
  for i in range(image.shape[0]):
    for j in range(image.shape[1]):
      mean = get_mean(i, j, image, n)
      filtered[i,j] = mean
  return filtered

# Function to apply median filter on an image
def median_filter(image, n):
  filtered = np.zeros(image.shape)
  for i in range(image.shape[0]):
    for j in range(image.shape[1]):
      median = get_median(i, j, image, n)
      filtered[i,j] = median
  return filtered

# Function to apply gaussian filter on an image
def gaussian_filter(image):
  filtered = np.zeros(image.shape)
  filter = np.array([[1,2,1],[2,4,2],[1,2,1]])/16
  for i in range(image.shape[0]):
    for j in range(image.shape[1]):
      image_pixels = np.zeros((3,3))
      image_pixels[0,0] = image[i-1, j-1] if (i-1>=0 and j-1>=0) else 0
      image_pixels[0,1] = image[i-1, j] if (i-1>=0) else 0
      image_pixels[0,2] = image[i-1, j+1] if (i-1>=0 and j+1<image.shape[1]) else 0
      image_pixels[1,0] = image[i, j-1] if (j-1>=0) else 0
      image_pixels[1,1] = image[i,j]
      image_pixels[1,2] = image[i, j+1] if (j+1<image.shape[1]) else 0
      image_pixels[2,0] = image[i+1, j-1] if (i+1<image.shape[0] and j-1>=0) else 0
      image_pixels[2,1] = image[i+1, j] if (i+1<image.shape[0]) else 0
      image_pixels[2,2] = image[i+1, j+1] if (i+1<image.shape[0] and j+1<image.shape[1]) else 0
      filtered[i,j] = np.sum(np.multiply(filter, image_pixels))
  return filtered

In [31]:
# Function to quantify image performance by using Peak Signal to Noise Ratio
def psnr(noisy, reconstructed):
  mse = np.mean((noisy - reconstructed) ** 2)
  if mse==0:
    return 100
  max_pixel = 255
  psnr = 20*log10((max_pixel)/sqrt(mse))
  return psnr

# CODE

## Reading and viewing the original image

In [ ]:
original_image = cv2.imread("1_Original.png",0)
cv2_imshow(original_image)

## Question 1: Geometric transformations on the original image

In [35]:
# Translate the image by 5.5 pixels and 4.4 pixels in x and y directions respectively
translated = translate(original_image, 5.5, 4.4)
cv2.imwrite("2_Translated.png", translated);

# Rotate the image by 35 degrees (CW)
rotated_35 = rotate(original_image, 35)
cv2.imwrite("2_Rotated_35.png", rotated_35);

# Rotate the image by 125 degrees (CC5)
rotated_125_ = rotate(original_image, -125)
cv2.imwrite("2_Rotated_-125.png", rotated_125_);

# Scale the image by a factor of 0.4
scaled_0_4 = scale(original_image, 0.4)
cv2.imwrite("2_Scaled_0_4.png", scaled_0_4);

# Scale the image by a factor of 1.4
scaled_1_4 = scale(original_image, 1.4)
cv2.imwrite("2_Scaled_1_4.png", scaled_1_4);

## Question 2: Contrast Enhancement using Histogram Equalization

In [38]:
# Plotting the histogram of the original image
original_pdf = histogram(original_image)
fig = plt.figure(figsize = (30, 10))
plt.bar(range(len(original_pdf)), original_pdf, width=0.5)
plt.savefig("2_Original_pdf")
plt.close()

# Plotting the CDF of the original image
original_cdf = get_cdf(original_pdf)
fig = plt.figure(figsize = (30, 10))
plt.bar(range(len(original_pdf)), original_cdf, width=0.5)
plt.savefig("2_Original_cdf")
plt.close()

# Obtaining the contrast enhanced image
hist_eq = hist_equalization(original_image, original_cdf)
cv2.imwrite("2_Hist_eq.png", hist_eq);

# Plotting the PDF of the contrast enhanced image
contrast_pdf = histogram(hist_eq)
fig = plt.figure(figsize = (30, 10))
plt.bar(range(len(original_pdf)), contrast_pdf, width=0.5)
plt.savefig("2_Contrast_pdf")
plt.close()

# Plotting the CDF of the contrast enhanced image
contrast_cdf = get_cdf(contrast_pdf)
fig = plt.figure(figsize = (30, 10))
plt.bar(range(len(original_pdf)), contrast_cdf, width=0.5)
plt.savefig("2_Contrast_cdf")
plt.close()

## Question 3: Types of Filters and Noises

In [ ]:
# To test a single example of CLAHE
# Image and filter names can be changed 
original_image = cv2.imread("Original_2.png", 0)
noisy = add_noise(original_image, "salt_and_pepper", 100)
cv2.imwrite("noisy.png", noisy)
filtered = gaussian_filter(noisy)
cv2.imwrite("filtered.png", filtered)
f = clahe(filtered, 9, 0.75)
cv2.imwrite("clahe_filtered.png", f)
print (psnr(noisy, f))

In [ ]:
# To test all possible combinations
Noises = ["salt_and_pepper", "gaussian"]
Variances = [25, 100]
Filters = ["Mean", "Median", "Gaussian"]
Neighbourhood = [4, 8]
clahe_filter_size = [3, 7, 9]
clip_limit = [0.75, 0.6]

for noise in Noises:
  for var in Variances:
    for filter in Filters:
      noisy = add_noise(original_image, noise, var)
      name = noise + "_" + str(var) + ".png"
      cv2.imwrite(name, noisy)
      if filter == "Mean":
        for n in Neighbourhood:
          filtered = mean_filter(noisy, n)
          name = noise+"_" + str(var)+"_" + "mean"+"_" + str(n)+".png"
          cv2.imwrite(name, filtered)
          for clahe_filter in clahe_filter_size:
            for limit in clip_limit:
              filtered_clahe = clahe(filtered, clahe_filter, limit)
              name = noise+"_" + str(var)+"_" + "mean"+"_" + str(n)+ "_" + str(clahe_filter) + "_" + str(limit) + ".png"
              cv2.imwrite(name, filtered_clahe)
              print (name, psnr(filtered_clahe, noisy))
      elif filter=="Median":
        for n in Neighbourhood:
          filtered = median_filter(noisy, n)
          name = noise+"_" + str(var)+"_" + "median"+"_" + str(n)+".png"
          cv2.imwrite(name, filtered)
          for clahe_filter in clahe_filter_size:
            for limit in clip_limit:
              filtered_clahe = clahe(filtered, clahe_filter, limit)
              name = noise+"_" + str(var)+"_" + "median"+"_" + str(n)+ "_" + str(clahe_filter) + "_" + str(limit) + ".png"
              cv2.imwrite(name, filtered_clahe)
              print (name, psnr(filtered_clahe, noisy))
      elif filter=="Gaussian":
        filtered = gaussian_filter(noisy)
        name = noise + "_" + str(var) + "_" + gaussian + ".png"
        cv2.imwrite(name, filtered)
        for clahe_filter in clahe_filter_size:
          for limit in clip_limit:
            filtered_clahe = clahe(filtered, clahe_filter, limit)
            name = noise+"_" + str(var)+"_" + "gaussian"+"_" + str(clahe_filter) + "_" + str(limit) + ".png"
            cv2.imwrite(name, filtered_clahe)
            print (name, psnr(filtered_clahe, noisy))